# CONSTRUCCIÓN DE LOS "FOOTPRINTS"

Construimos los footprints temporales de cada cliente a partir de sus transacciones, de tal manera que agrupamos en un intervalo de d días.


Dado:

    Sesion de TX
        s = {cliente, timestamp, Monto}
    
    Cada cliente tiene una secuencia de sesiones de TX (S)
        S = { s1, s2 , s3, ..., sn}                n: # de sesiones de cada cliente
        
        

In [1]:
# LIBRERIAS
import numpy as np
import datetime
from datetime import date
import json
import pylab
import pandas as pd
import matplotlib.pyplot as plt
import os, sys

## Preparacion de datos

### Cargando datos

In [2]:
data_original = pd.read_csv("./data/muestra.csv", encoding = "iso-8859-1")
data_original.head()

,CO_ID,BILLCYCLE,NUMCODPLANTARIFARIO,F_ACT,F_TRAFICO,HORA,GB_TOTAL,NO_FREE,FREE,BANCOS,...,SPOTIFY_PI,APPLEM_PI,SOUNDC_PI,FACE_PI,MAIL_PI,WHATS_RO_I,WHATS_RO_IF,WAZE_RO,EMAIL_RO,WHARS_RO2
0,27782393,3.0,1981,11/11/2016,02/08/2018,Noche,0.0000,0.0000,0.0000,0.0,...,0.0,0,0.0,0,0,0.0,0.0,0.0,0,0.0
1,41901365,5.0,1982,20/01/2018,02/08/2018,Mañana,0.0411,0.0301,0.0007,0.0,...,0.0,0,0.0,0,0,0.0,0.0,0.0,0,0.0
2,31801426,5.0,1981,17/04/2017,02/08/2018,Madrugada,0.0138,0.0078,0.0014,0.0,...,0.0,0,0.0,0,0,0.0,0.0,0.0,0,0.0
3,41410806,4.0,1982,05/01/2018,02/08/2018,Madrugada,0.0799,0.0603,0.0078,0.0,...,0.0,0,0.0,0,0,0.0,0.0,0.0,0,0.0
4,42886287,5.0,1981,20/02/2018,02/08/2018,Mañana,0.2182,0.0096,0.0000,0.0,...,0.0,0,0.0,0,0,0.0,0.0,0.0,0,0.0


### Preparacion de datos

In [3]:
data = data_original[['CO_ID','BILLCYCLE','F_TRAFICO','HORA','GB_TOTAL']]
data.head(5)

,CO_ID,BILLCYCLE,F_TRAFICO,HORA,GB_TOTAL
0,27782393,3.0,02/08/2018,Noche,0.0000
1,41901365,5.0,02/08/2018,Mañana,0.0411
2,31801426,5.0,02/08/2018,Madrugada,0.0138
3,41410806,4.0,02/08/2018,Madrugada,0.0799
4,42886287,5.0,02/08/2018,Mañana,0.2182


## Definicion de variables

### mccgs

In [4]:
# DEFINIMOS MCCGS: PARA CREAR UN FOOTPRINT DINAMICO
billcycle = ((data.groupby(['BILLCYCLE'], as_index=False, sort=True)).count())['BILLCYCLE']
billcycle

0    1.0
1    3.0
2    4.0
3    5.0
Name: BILLCYCLE, dtype: float64

### Clientes

In [5]:
# DEFINIMOS LA LISTA DE CLIENTES
clientes =  data.groupby('CO_ID').CO_ID.count().index
clientes

Int64Index([ 2581717,  3132200,  3273844,  3478078,  3768670,  3877738,
             3903875,  3952329,  3954876,  3956151,
            ...
            46812929, 46813575, 46817874, 46822457, 46832405, 46842470,
            46845419, 46845790, 46853110, 46856084],
           dtype='int64', name='CO_ID', length=43675)

# FOOTPRINT PARA CADA MCCG

## Unidad de TXs temporales (U)

Resume un conjunto de TXs en un periodo de tiempo **“tau”**.
<div>Los footprints que creamos será la agrupación de todas las transacciones en "tau" de cada cliente, y representará una unidad de comportamiento con 3 dimensiones.</div> 


Donde:
1.  **tau** = 1 semana
2.  dimensión 1:    **d**   = 7 días
3.  dimensión 2:	**t**   = 4 turnos	 
4.  dimensión 3:    **m**   = # de mccgs en nuestra data.

    
Para cada cliente:
    
    Entrada:   S = {s1, ..., sn}
    Salida:    SS = {U(1), U(2), U(3), ...,U(m)}     donde m <=n

### Funciones de apoyo

Definimos funcion para generar los footprint (U) de un usuario, con los parametros:

1. <div> **user**:  Dataset original filtrado para un cliente determinado</div> 
2. <div> **monto**: Indica si los footprint seran generados con la suma de los el numero de TXs (False) o la suma de los montos por cada TX (True)</div> 

In [13]:
def procesar_u(user, monto = False):    
    uid=list(user['CO_ID'])[0]              # Cliente_id
    years = set(list(user['año']))              # Lista los años en que se tiene TXs registradas
    anni = {year:{} for year in list(years)}    # definimos anni como una lista 
    
    # para cada fila, es decir, cada TXs del cliente)
    for dat in  range(0,len(user)):
        fecha = user.iloc[dat]['F_TRAFICO']
        print(fecha)
        año = fecha[7:]
        mes = fecha[4:5]
        dia = fecha[1:2]
        print(año,mes,dia)
        week=str(datetime.datetime(año,user.iloc[dat]['mes'],user.iloc[dat]['dia']).isocalendar()[1])
        if len(week)==1:
            week = '0'+week
        weekday=datetime.datetime(año,user.iloc[dat]['mes'],user.iloc[dat]['dia']).weekday()
        turn = user.iloc[dat]['HORA']
        mccg = user.iloc[dat]['BILLCYCLE']
        
        # Si la semana no existe en el año
        if not(week in anni[año]):
            anni[año][week] = {}
        # Si el mccg no existe en la semana y año
        if not (mccg in anni[año][week]):
            anni[año][week][mccg]={}  #NUMERO DE MCCGs VARIABLES
        # Si el turno no existe en el mccg,semana y año
        if not (turn in anni[año][week][mccg]):
            anni[año][week][mccg][turn]=np.array([0]*7)  #CUATRO TURNOS
            
        if monto:
            # suma montos "importancia por gastos"
            anni[año][week][turn][weekday]+=user.iloc[dat]['amount_sol']
        else:
            # suma cantidades "importancia por compras"
            anni[año][week][mccg][turn][weekday]+=user.iloc[dat]['quantity'] 
            
    return uid,anni

### Procesando U

Generamos en formato json el footprint de cada cliente
donde el los índices son: **cliente_id**, **año**, **semana**, **mccg**, **turno** conteniendo un **array[0,...,6]** con los dias.



In [14]:
##################################################
#        Procesando U de cada CLIENTE
##################################################
    
profiles={}           # Creamos lista de prefiles
contador=0 
print("Number of rows "+str(len(data))) 

# Para cada cliente
for cliente in clientes:
    cliente_i= data[data['CO_ID'] == cliente]       # filtramos dataset solo para el cliente i
    results=procesar_u(cliente_i, monto=False)          # procesamos u del usuario i
    #profiles[results[0]]=results[1]                     # cargamos lista de indice "uid" con la data del cliente(json)
    

    contador += 1
    if contador % 100000 == 1:
        print("vamos en el ",contador)
        
cliente_i

Number of rows 925607


KeyError: 'año'

In [ ]:
# profiles

Creamos la cabecera dinámica donde se guardaran todos los footprints generados

In [ ]:
cabecera = 'customer_id,year,week,profile_id,mccg,turn,size'
for i in range(len(mccgs)):
    for j in range(4):                # numero de turnos
        for k in range(7):            # numero de dias
            cabecera = cabecera+','+'m'+str(mccgs[i])+'t'+str(j)+'d'+str(k)
cabecera = cabecera+'\n'

In [ ]:
outfile='./RESULTADOS/U'           # Indicamos archivo de salida 
individual_footprint="%s.footprint" %(outfile)
fw=open(individual_footprint,'w')  

fw.write(cabecera)                    # Escribimos la cabecera
footprints=0 

# Guardamos el Json como un dataframe
# Para cada uid (cliente)
for uid in profiles:   
    profile_id=0
    # En cada año
    for year in profiles[uid]:       
        # Por cada semana
        for week in profiles[uid][year]:                        
            temp=np.zeros(4*7*len(mccgs))                      # creamos tensor con 0s
            
            for mccg in profiles[uid][year][week]:             # cargamos los mccgs
                pos_mccg = list(mccgs).index(mccg)             # posicion del mccg
                temp2 = np.zeros(4*7)                          # creamos vector temporal turnos por dias con 0s
                for turn in profiles[uid][year][week][mccg]:   # cargamos turnos
                    pos_turn = turn                            # posicion del turno
                    
                    d=profiles[uid][year][week][mccg][turn]    # Dias como array        
                    
                    for k in range(pos_turn*7,(pos_turn+1)*7): # cargamos dias
                        temp2[k] += d[k-(pos_turn*7)]          # posicion de cada dia en el vector temporal
                
                for j in range(pos_mccg*28,(pos_mccg+1)*28):
                    temp[j] = temp2[j-(pos_mccg*28)]           # vector temporal es cargado en el tensor
                
            # Escribimos los datos del primer comportamiento (Tensor)    
            txt = ''+str(uid)+','+str(year)+','+str(week)+','+str(profile_id)+','+str(mccg)+','+str(turn)+','+str(sum(temp))
            for i in range(len(temp)):
                txt = txt +','+str(temp[i])
            fw.write(txt +'\n')

            profile_id += 1    # perfil cambia cada unidad de fecha diferente (eg. cada semana != es un perfil) de cada cliente
            footprints += 1
    fw.flush()
fw.close()
print ("number of footprint: "+str(footprints))